In [1]:
# desc

In [2]:
import json

import requests

# import the smartpynector module
from NBs import smartpynector as sp


In [3]:
# Defining constants
# Knowledge engine REST API URL
KNOWLEDGE_ENGINE_URL = "http://127.0.0.1:8280/rest"
THERMOSTAT_API_URL = "http://127.0.0.1:8000/thermostat"

In [4]:
# Defining the Smart Connector
smart_connector = sp.SmartConnector(
    knowledgeBaseId=THERMOSTAT_API_URL,
    knowledgeBaseName="thermostat 1",
    knowledgeBaseDescription="thermostat 1",
    reasonerEnabled=False,
)

In [5]:
sp.create_smart_connector(smart_connector, KNOWLEDGE_ENGINE_URL)

2023-05-03 18:22:50 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-03 18:22:53 DEBUG http://127.0.0.1:8280 "POST /rest/sc HTTP/1.1" 200 0
2023-05-03 18:22:53 INFO Request successful
2023-05-03 18:22:53 DEBUG 


<Response [200]>

In [6]:
answer_ki_id = sp.register_knowledge_interaction(
    knowledge_interaction_type="AnswerKnowledgeInteraction",
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    prefixes={
        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "saref": "https://w3id.org/saref#",
    },
    graph_pattern='''?meas rdf:type saref:Measurement .
         ?meas saref:hasValue ?temp .
         ?meas saref:isMeasuredIn saref:TemperatureUnit .
         ?meas saref:hasTimestamp ?timestamp .
         ?meas saref:isMeasurementOf ?room_id .
         ?meas saref:relatesToProperty saref:Temperature .
         ?meas saref:measurementMadeBy ?device_id .''',
    headers={
        "Content-Type": "application/json",
        "knowledge-Base-Id": THERMOSTAT_API_URL
    },
)["knowledgeInteractionId"]

answer_ki_id

2023-05-03 18:22:53 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-03 18:22:54 DEBUG http://127.0.0.1:8280 "POST /rest/sc/ki HTTP/1.1" 200 110
2023-05-03 18:22:54 INFO Request successful
2023-05-03 18:22:54 DEBUG {"knowledgeInteractionId":"http://127.0.0.1:8000/thermostat/interaction/3a2a44f2-76d3-44f1-a985-ea9573aa3e46"}


'http://127.0.0.1:8000/thermostat/interaction/3a2a44f2-76d3-44f1-a985-ea9573aa3e46'

In [7]:
thermostat_actual_temperature = requests.get(THERMOSTAT_API_URL + '/actual_temperature').json()
thermostat_actual_temperature

2023-05-03 18:22:54 DEBUG Starting new HTTP connection (1): 127.0.0.1:8000
2023-05-03 18:22:54 DEBUG http://127.0.0.1:8000 "GET /thermostat/actual_temperature HTTP/1.1" 200 2


20

In [8]:
# %%timeit
sp.perform_answer_knowledge_interaction(
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    knowledge_base_id=THERMOSTAT_API_URL,
    knowledge_interaction_id=answer_ki_id,
    answer_binding_set=[{
        "meas": f'<{THERMOSTAT_API_URL}/measurements/1>',
        "temp": f"{thermostat_actual_temperature}",
        "timestamp": f'"{sp.get_timestamp_now()}"',  # ISO 8601 format
        "room_id": f'"{THERMOSTAT_API_URL}/rooms/1"',
        "device_id": f'"{THERMOSTAT_API_URL}/devices/1"',
    }]
)

2023-05-03 18:22:54 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-03 18:22:56 DEBUG http://127.0.0.1:8280 "GET /rest/sc/handle HTTP/1.1" 200 209
2023-05-03 18:22:56 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-03 18:22:56 DEBUG http://127.0.0.1:8280 "POST /rest/sc/handle HTTP/1.1" 200 0


<Response [200]>